In [1]:
import math
import os
import openpyxl
import platform
import numpy as np
import pandas as pd

In [2]:
if platform.platform()[:3].lower() == 'mac':
    mac_m1 = True
elif platform.platform()[:3].lower() == 'win':
    mac_m1 = False
    
if mac_m1:
    path = '/Users/rainyseason/winston/Workspace/python/Pycharm Project/sinewave_analyze/Evaluation/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv'
else:
    path = 'D:/winston/OneDrive - (주)필드큐어/data_analyze/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv/'
    
evaluation_control_file = 'eval_control.xlsx'

In [16]:
path_excel = '/Users/rainyseason/winston/Workspace/python/Pycharm Project/sinewave_analyze/Evaluation/tek_excel/'

In [17]:
files = os.listdir(path_excel)
files = [file for file in files if '.DS' not in file and '~' not in file]
files.sort()
for file in enumerate(files):
    print(file)

(0, 'Ch3 Ch4')
(1, 'tek0000 08Ch3xlsx')
(2, 'tek00000 08Ch3xlsx')
(3, 'tek00001 08Ch3xlsx')
(4, 'tek00002 08Ch3xlsx')
(5, 'tek00003 08Ch3xlsx')
(6, 'tek00004 08Ch3xlsx')
(7, 'tek00005 08Ch3xlsx')
(8, 'tek00006 08Ch3xlsx')
(9, 'tek00007 08Ch3xlsx')
(10, 'tek00008 08Ch3xlsx')
(11, 'tek00009 08Ch3xlsx')
(12, 'tek0001 08Ch3xlsx')
(13, 'tek00010 08Ch3xlsx')
(14, 'tek00011 08Ch3xlsx')
(15, 'tek00012 08Ch3xlsx')
(16, 'tek00013 08Ch3xlsx')
(17, 'tek00014 08Ch3xlsx')
(18, 'tek00015 08Ch3xlsx')
(19, 'tek00016 08Ch3xlsx')
(20, 'tek00017 08Ch3xlsx')
(21, 'tek00018 08Ch3xlsx')
(22, 'tek00019 08Ch3xlsx')
(23, 'tek00020 08Ch3xlsx')
(24, 'tek00021 08Ch3xlsx')
(25, 'tek00022 08Ch3xlsx')
(26, 'tek00023 08Ch3xlsx')
(27, 'tek00024 08Ch3xlsx')
(28, 'tek00025 08Ch3xlsx')
(29, 'tek00026 08Ch3xlsx')
(30, 'tek00027 08Ch3xlsx')
(31, 'tek00028 08Ch3xlsx')
(32, 'tek00029 08Ch3xlsx')
(33, 'tek00030 08Ch3xlsx')
(34, 'tek00031 08Ch3xlsx')
(35, 'tek00032 08Ch3xlsx')
(36, 'tek00033 08Ch3xlsx')
(37, 'tek00034 08Ch3xlsx

In [15]:
path_ch3 = path_excel
path_ch4 = path